##  FastAPI DEPENDS 

Dependency  means we can inject functions on some class

for examples 
 engine is a part of car 
 in traditional system -> 
```   class car 
           engine defines under class car 

 class Engine:
    def start(self):
        return "Vroom!"

class Car:
    def __init__(self):
        # The Car is "Hard-Coupled" to the Engine
        self.engine = Engine() 

    def drive(self):
        print(self.engine.start()) ``` 


problem : if we want to change the engine to Electrict enginer or turboengine , we have to modify the car class code itself ... its voilates Open closed principle (coded should be open for extension but closed for modification)




## in dependency injection

In DI, the Car doesn't care how the engine is made; it only cares that it receives something that acts like an engine.

In [3]:
class Engine:
    def start(self):
        return "Standard Engine started"

class ElectricEngine:
    def start(self):
        return "Silent Electric Motor started"

class Car:
    def __init__(self, engine): # Injecting the dependency
        self.engine = engine

    def drive(self):
        print(self.engine.start())

# --- Usage ---
std_engine = Engine()
tesla_engine = ElectricEngine()

# We "Inject" the dependency at runtime
my_car = Car(std_engine)
my_electric_car = Car(tesla_engine)

***********************************************************************

class Engine:
    def start(self):
        return "Vroom!"

class Car:
    def __init__(self):
        # The Car is "Hard-Coupled" to the Engine
        self.engine = Engine() 

    def drive(self):
        print(self.engine.start())

In [ ]:
import asyncio
import uvicorn
from fastapi import FastAPI, Depends

# --- 1. CLEANUP PREVIOUS RUNS ---
if 'server' in globals():
    server.should_exit = True
    # Give it a moment to release the port
    await asyncio.sleep(1) 

# --- 2. DEFINE YOUR APP ---
app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello from the new cell!"}

# --- 3. START THE SERVER ---
config = uvicorn.Config(app, port=8000, host="127.0.0.1")
server = uvicorn.Server(config)

loop = asyncio.get_running_loop()
loop.create_task(server.serve())

print("Server updated and running on http://127.0.0.1:8000")

INFO:     Started server process [469902]
INFO:     Waiting for application startup.


Server updated and running on http://127.0.0.1:8000


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51994 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52000 - "GET /favicon.ico HTTP/1.1" 404 Not Found
